# 扩散模型加速采样算法《Denoising Diffusion lmplicit Models》论文原理与PyTorch源码解读

来自b站up主deep_thoughts 合集【PyTorch源码教程与前沿人工智能算法复现讲解】

P_64_扩散模型加速采样算法DDIM论文精讲与PyTorch源码逐行解读：

https://www.bilibili.com/video/BV1JY4y1N7dn/?spm_id_from=333.788&vd_source=18e91d849da09d846f771c89a366ed40

## 本期直播主要内容
* 指出DDPM中的$L_{simple}$与扩散过程的联合分布具体形式无直接关系，因此训练$L_{simple}$其实就相当于训练了一些列潜在的扩散模型
* 构造了一种更具一般性的非马尔科夫的扩散过程，并让其满足边缘分布不变以重复利用训练好的DDPM模型。
* 构造了更具一般的采样算法，涵盖DDPM和DDIM
* 指出可以用respacing来减少采样步骤，并发现DDIM可以以5倍少的步骤来获得跟DDPM一样的采样质量
* 对improved diffusion中的respacing和ddim_sample等函数源码进行补充讲解

## notebook目录

* 1 摘要与引文
* 2 DDPM原理回顾
* 3 DDPM的损失函数特点
* 4 设计非马尔可夫链的前向扩散过程
* 5 对比非马尔科夫扩散后验分布与DDPM马尔科夫扩散的后验分布
* 6 非马尔科夫扩散逆过程的采样
* 7 一种特殊的采样--DDIM(含蓄的概率扩散模型)
* 8 $L_{simple}$的特殊性质带来一种加速采样的技巧--respacing
* 9 对比论文中DDIM和DDPM的respacing加速采样效果·
* 10 DDPM loss函数、DDIM sample函数以及加速采样的respacing函数的PyTorch源码(回顾)讲解

## 本期直播参考文献和参考视频
DDPM论文、DDIM论文以及54、58、62、63期视频教程。

# 1. 摘要与引文

![](./img/P64_1.png)

![](./img/P64_2.png)

![](./img/P64_3.png)

# 2. DDPM原理回顾

![](./img/P64_4.png)

![](./img/P64_5.png)

# 3. DDPM的损失函数特点

## DDPM的损失函数$L_{simple}$只依赖于边缘分布，而不直接依赖于联合分布

![](./img/P64_6.png)

在推导出$L_{simple}$(或$L_\gamma$)过程中，我们没有用到$q(x_{1:T}|x_0)$的具体形式，只是基于贝叶斯公式和$q(x_t,|x_{t-1},x_0)$、$q(x_t|x_0)$表达式。

在训练DDPM所用到的$L_{simple}$ loss中，我们甚至都没有采用跟$q(x_t,|x_{t-1},x_0)$相关的系数，而是直接选择将预测噪音的权重设置为1。由于噪音项是来自$q(x_t|x_0)$的采样，因此，DDPM的目标函数其实只由$q(x_t|x_0)$表达式决定。这其实也证实了$L_{simple}$与score-matching之间的联系。

换句话说，只要$q(x_t|x_0)$已知并且是高斯分布的形式，那么我们就可以用DDPM的预测噪音的目标函数$L_{simple}$ 来训练模型。

于是，槽点来了!!!在DDPM中，基于马尔科夫性质我们认为$q(x_t,|x_{t-1},x_0)=q(x_t|x_{t-1})$，那么如果是服从非马尔科夫性质呢?$q(x_t,|x_{t-1},x_0)$是不是具有更一般的形式?以及，只要我们保证$q(x_t|x_0)$的形式不变，那么我们可以直接复用训好的DDPM,只不过使用新的概率分布来进行逆过程的采样。

接下来作者给出了一种非马尔科夫性质的前向扩散讨程的公式以及后验概率分布的表达式。而该后验概率分布恰好满足DDPM中的边缘分布$q(x_t|x_0)$。

# 4. 设计非马尔可夫链的前向扩散过程

![](./img/P64_7.png)

## 接下来证明$q_{\sigma}(x_t|x_0)$

## 首先，回顾一下边缘高斯分布与条件高斯分布之间的推理关系

![](./img/P64_8.png)

## 其次，回顾一下高中阶段学过的数学归纳法证明

![](./img/P64_9.png)

![](./img/P64_10.png)

## 最后，基于条件概率公式+高斯分布的性质+数学归纳法来证明$q_{\sigma}(x_t|x_0)$

![](./img/P64_11.png)

![](./img/P64_12.png)

## 到此，我们已经证明了给定下式的情况下，边缘分布

# 5. 对比非马尔科夫扩散后验分布与DDPM马尔科夫扩散的后验分布

## DDPM马尔科夫扩散的后验分布如下

![](./img/P64_13.png)

# 6. 非马尔可夫扩散逆过程的采样

![](./img/P64_14.png)

## 下面证明$J_{\sigma}=L_{\gamma}+C$

![](./img/P64_15.png)

## 优化$L_1$同样也是在优化$J_{sigma}$

# 7.一种特殊的采样--DDIM(含蓄的概率扩散模型)

![](./img/P64_16.png)

# 8. $L_1的特殊性质带来一种加速采样技巧--respacing$

![](./img/P64_17.png)

# 9. 对比DDIM和DDPM的respacing加速采样效果

![](./img/P64_18.png)

![](./img/P64_19.png)

# 10 DDPM Ioss函数、DDIM sample函数以及respacing函数的PyTorch源码
## 主要围绕OpenAl的improved diffusion项目源码讲解

代码仓库：https://github.com/openai/improved-diffusion

代码讲解见视频